In [98]:
import math
import random
import numpy as np
import copy

In [99]:
def fungUji(x, y):
    return x**2 + y**2 + 25*(np.sin(x)**2 + np.sin(y)**2)

In [100]:
def levy(d):
    lamda = 1.5
    sigma = (math.gamma(1 + lamda) * math.sin(math.pi * lamda / 2) / (
    math.gamma((1 + lamda) / 2) * lamda * (2 ** ((lamda - 1) / 2)))) ** (1 / lamda)
    # sigma = 0.6965745025576968
    u = np.random.randn(1, d) * sigma
    v = np.random.randn(1, d)
    step = u / abs(v) ** (1 / lamda)
    return 0.01 * step

In [101]:
def simple(s, lb, ub, d):
    ns_tmp = s
    for i in range(0, d):
        if ns_tmp[i] < lb:
            ns_tmp[i] = lb
        if ns_tmp[i] > ub:
            ns_tmp[i] = ub
    return ns_tmp

In [119]:

#Sol = Posisi kelelawar
#S = Posisi kelelawar sementara
#A dan r masih konstan
def baFpa(n, A, r):
#     n => population of bats
#     A => loudness
#     r = pulse rate
    
    Qmin = 0 # lowest frequency
    Qmax = 2 # highest frequency
    
    fmin_FPA = 0.0
    fmin_BAT = 0.0
    
    #Loop parameter
    tol = 10 #loop limit
    N_iter = 0     #number of iterations
    
    #search space dimension
    d = 2
    
    #initial array of values
    Q = np.zeros(n) #frequency each bat
    v = np.zeros((n, d)) #the speed of each bat in each dimension
    
    #assignment of initial values in bat populations (solution)
#     Sol = np.zeros((n, d))
    Sol = np.random.randint(5, size=(n, d))
    S = np.random.randint(5, size=(n, d))
    Fitness = np.full(n, (99.99))
    
    #initial best solution
    initialFungUji = []
    for i in range(len(Sol)):
        initialFungUji.append(fungUji(Sol[i, 0], Sol[i, 1]))
        
    fmin = min(initialFungUji)
    fminIndex = initialFungUji.index(fmin)
    best = Sol[fminIndex, :]
    
    #The application of bat algorithm
    while N_iter < tol:
        
        #loops imposed all bats
        for i in range(n):
            beta = np.random.rand()
            Q[i] = Qmin + (Qmax-Qmin) * beta
            v[i, :] = v[i, :] + (Sol[i, :] - best) * Q[i] #*
            S[i, :] = Sol[i, :] + v[i, :]
            
             #pulse rate effect
            alpha = 0.01
            if np.random.rand() > r:
                S[i, :] = best + alpha * np.random.randn(1, d)
        
            S[i,] = simple(S[i,], 2.0, 99.99, 2)
            x = S[i, 0]
            y = S[i, 1]
            Fnew = fungUji(x, y)
            
            #Flying randomly
#             v[i, :] = v[i, :] + (Sol[i, :] - best) * Q[i] #*
#             S[i, :] = Sol[i, :] + v[i, :]
        
            if Fnew <= Fitness[i] and np.random.rand() < A:
                Sol[i, :] = S[i, :]
                Fitness[i] = Fnew
                
                #increase r[i] reduce A[i]
                #A[i] = alpha * A[i]
                #r[i] = r[i] * (1- math.exp(-1*gamma * N_iter)
                
            #=======================FPA ALGORITHM===================================
            S_FPA = Sol.copy()
            Sol_FPA = Sol.copy()
            Fnew_FPA = Fnew.copy()
            Fitness_FPA = Fitness.copy()
    
            for t in range(0, N_iter):
                for i in range(0, n):
                    p=0.8-0.7*t/N_iter
                    if np.random.random() < p:
                        L = levy(d)
                        S_FPA[i,] = Sol_FPA[i,] + L * (Sol_FPA[i,] - best)
                    else:
                        epsilon = np.random.random_sample()
                        #jk -> Find random flowers in the neighbourhood
                        jk = np.random.permutation(n)
                        S_FPA[i,] = S_FPA[i,] + epsilon * (Sol_FPA[jk[0],] - Sol_FPA[jk[1],])

                    S[i,] = simple(S[i,], 2.0, 99.99, 2)
                    x = S_FPA[i, 0]
                    y = S_FPA[i, 1]
            
                    Fnew_FPA = fungUji(x, y)
                    if Fnew_FPA <= Fitness_FPA[i]:
                        Sol_FPA[i,] = S_FPA[i,]
                        Fitness_FPA[i] = Fnew_FPA

            #=======================END FPA ALGORITHM===============================
            #=======================Start Find Best Position==============================
            fmin_FPA = np.amin(Fitness_FPA)
            fminFPAIndex = np.where(Fitness_FPA == np.amin(Fitness_FPA))
            fmin_BAT = np.amin(Fitness)
            fminBATIndex = np.where(Fitness == np.amin(Fitness))
            
            #update the smallest / best test function value
            if fmin_FPA < fmin_BAT:
                best = S_FPA[fminFPAIndex[0][0], :].copy()
                fmin = fmin_FPA.copy()
            else :
                best = S[fminBATIndex[0][0], :].copy()
                fmin = fmin_BAT.copy()
#             print(best)
#             print(fmin)
            #=======================End Find Best Position==============================
        print(best)
        print(fmin)
        N_iter = N_iter + 1
#     print(best)
#     print(fmin)

In [120]:
baFpa(4, 0.25, 0.5)

[2 2]
49.3410905215903
[0 3]
9.497871416870424
[0 2]
24.67054526079515
[0 1]
18.70183545683928
[1 2]
0.0
[0 0]
0.0
[1 0]
18.70183545683928
[0 0]
0.0
[0 1]
0.0
[0 1]
0.0
